In [ ]:
"""
%pip install pymupdf
%pip install pinecone
%pip install groq
%pip install --upgrade langchain
%pip install langchain-community
%pip install pypdf
%pip install langchain-community
"""

In [1]:
import fitz  # PyMuPDF
from pinecone import Pinecone, ServerlessSpec
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
import torch

In [2]:
# Selección del device
device =  'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device) # Expected: ‘cuda’ if Linux else ‘mps’ if MacOS

mps


## 2 - Carga del CV y obtención de los embeddings


In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader

# Leer documentos
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

doc=read_doc("./")

In [ ]:
"""
from docx import Document

# Abre el archivo .docx y lo devuelve como texto
def open_content(path):
    
    # Abrir el archivo .docx
    doc = Document(path)
    
    # Extraer todo el texto del documento
    content = ''
    for para in doc.paragraphs:
        content += para.text + '\n'  # Concatenar cada párrafo con salto de línea
    
    return content

# Leer y procesar el contenido para añadirlo a la colección en Chroma
doc = open_content('Silva_Victor_CV.docx')
"""

In [4]:
doc

[Document(metadata={'source': 'Silva_Victor_CV.pdf', 'page': 0}, page_content='VíctorDavidSilvawww .link edin.com/in/vict or-da vid-silv a Mendoza,Argentina•vic.silva.1994@gmail.com•+5492616104445ExperienceIMPSA Mendoza,ArgentinaIAEngineer July2024–Present● Developingafaultdetectionsystemforsolarpanelsusingartificialintelligence.Thermalimagesarecollectedbyadrone,andmalfunctionsareidentifiedandcategorizedbytypeusingcomputervision.\nAutomationEngineer November 2021–Present● DevelopedanddesignedaProgrammableLogicController(PLC)programandHuman-MachineInterface(HMI)forabutterflyvalveatahydraulicpowerplant.● DevelopedanddesignedProgrammableLogicController(PLC)program,Human-MachineInterface(HMI)andSupervisoryControlandDataAcquisition(SCADA)systemforalevelluffingcranesystemusingTIAPortal,fortheArgentinaNavy.● DevelopedanddesignedProgrammableLogicController(PLC)programandHuman-MachineInterface(HMI)foranoverheadcranesystemusingTIAPortal,foranuclearpowerplant.● Resolvedissuesinpowerplantsthroughr

In [5]:
print(type(doc))  # Verifica el tipo de `doc`
print(doc[0])     # Muestra el primer elemento para entender su estructura


<class 'list'>
page_content='VíctorDavidSilvawww .link edin.com/in/vict or-da vid-silv a Mendoza,Argentina•vic.silva.1994@gmail.com•+5492616104445ExperienceIMPSA Mendoza,ArgentinaIAEngineer July2024–Present● Developingafaultdetectionsystemforsolarpanelsusingartificialintelligence.Thermalimagesarecollectedbyadrone,andmalfunctionsareidentifiedandcategorizedbytypeusingcomputervision.
AutomationEngineer November 2021–Present● DevelopedanddesignedaProgrammableLogicController(PLC)programandHuman-MachineInterface(HMI)forabutterflyvalveatahydraulicpowerplant.● DevelopedanddesignedProgrammableLogicController(PLC)program,Human-MachineInterface(HMI)andSupervisoryControlandDataAcquisition(SCADA)systemforalevelluffingcranesystemusingTIAPortal,fortheArgentinaNavy.● DevelopedanddesignedProgrammableLogicController(PLC)programandHuman-MachineInterface(HMI)foranoverheadcranesystemusingTIAPortal,foranuclearpowerplant.● Resolvedissuesinpowerplantsthroughremoteinterventions,offeringefficientsolutionstochal

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

def chunk_documents(documents, chunk_size=50, chunk_overlap=15):
    text_splitter = CharacterTextSplitter(
        separator=" ",#["\n", " "],  # Divide por líneas primero
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = []
    for doc in documents:
        # Divide el contenido del texto en fragmentos
        for chunk in text_splitter.split_text(doc.page_content): 
            chunks.append({
                "text": chunk,
                "metadata": doc.metadata #  # Mantén la fuente original
            })
    return chunks

# Divide los documentos en fragmentos
chunked_docs = chunk_documents(doc)

# Verifica el número de fragmentos generados
print(f"Total de fragmentos generados: {len(chunked_docs)}")


In [ ]:
chunked_docs[8]['text']

## Carga de documentos en la base de datos

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

# Configurar clave de API de Pinecone
PINECONE_API_KEY = os.getenv("pinecone_api_key")
pc = Pinecone(api_key=PINECONE_API_KEY)

In [11]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

index_name = 'cv'

In [ ]:
import time

# Crear índice si no existe
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=384,  # dimensionality of minilm
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)

time.sleep(1)
# view index stats
index.describe_index_stats()

In [13]:
# Configuración del modelo de embedding
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device=device)

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from sentence_transformers import SentenceTransformer
import os
from tqdm.auto import tqdm

# Leer documentos
def read_doc(directory):
    file_loader = PyPDFDirectoryLoader(directory)
    documents = file_loader.load()
    return documents

# Procesar documentos y generar embeddings
def process_docs(documents):
    texts = [doc['text'] for doc in documents]
    embeddings = embed_model.encode(texts, convert_to_tensor=False, show_progress_bar=True)
    return texts, embeddings


# Leer y procesar documentos
#directory = "./"  # Directorio donde está tu CV en formato PDF
#documents = read_doc(directory)
texts, embeddings = process_docs(chunked_docs)

# Subir los embeddings a Pinecone
ids = [f"doc_{i}" for i in range(len(texts))]  # Crear IDs únicos para los documentos
vectors = [{"id": doc_id, "values": emb, "metadata": {"text": text}}
           for doc_id, emb, text in zip(ids, embeddings, texts)]

# Subir los vectores al índice en lotes
batch_size = 5
for i in tqdm(range(0, len(vectors), batch_size)):
    batch = vectors[i:i + batch_size]
    index.upsert(batch)

# Ver estadísticas del índice
stats = index.describe_index_stats()
print(stats)

print(f"Generados {len(embeddings)} embeddings.")

### Prueba de Pinecone

In [ ]:
query = "Human machine interface"

# create the query vector
xq = embed_model.encode(query).tolist()

# now query
xc = index.query(vector=xq, top_k=5, include_metadata=True)
xc

## 5 - Implementación de un chatbot simple

In [16]:
import os
from groq import Groq

# Carga la clave de API de GROQ desde las variables de entorno
GROQ_API_KEY = os.getenv("groq_api_key")

# Crea el cliente de GROQ
client = Groq(
    api_key=GROQ_API_KEY,
)


In [ ]:
from groq import Groq

# Configurar Groq
GROQ_API_KEY = os.getenv("groq_api_key")
groq_client = Groq(api_key=GROQ_API_KEY)

def query_pinecone(index, query, top_k=5):
    """
    Consulta Pinecone y devuelve los documentos más relevantes.
    """
    # Crear el vector de la consulta
    query_vector = embed_model.encode(query).tolist()
    
    # Consultar Pinecone
    results = index.query(vector=query_vector, top_k=top_k, include_metadata=True)
    return results["matches"]

def create_prompt(query, pinecone_matches):
    """
    Crea un prompt para Groq utilizando la consulta y los documentos relevantes.
    """
    context = "\n".join([match["metadata"]["text"] for match in pinecone_matches])
    prompt = f"""
    Contexto relevante:
    {context}

    Pregunta:
    {query}

    Responde con información relevante al contexto y de forma concisa:
    """
    return prompt

def ask_groq(client, prompt, model="llama3-8b-8192"):
    """
    Envía el prompt a Groq y devuelve la respuesta.
    """
    response = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model
    )
    return response.choices[0].message.content

# Flujo principal
query = "Human machine interface" #"Human machine interface"
top_k = 5

# 1. Consultar Pinecone
matches = query_pinecone(index, query, top_k=top_k)

# 2. Crear el prompt para Groq
prompt = create_prompt(query, matches)

# 3. Enviar el prompt a Groq y obtener la respuesta
response = ask_groq(groq_client, prompt)

print("Respuesta de Groq:")
print(response)


In [ ]:
import streamlit as st

# Inicializa el historial de conversación en el estado de la sesión
if "conversation_history" not in st.session_state:
    st.session_state.conversation_history = []

def generate_response(input_text):
    # Agrega el mensaje del usuario al historial de conversación
    st.session_state.conversation_history.append({"role": "user", "content": input_text})

    # Genera la respuesta del chatbot utilizando el modelo LLaMA 3 y el historial de la conversación
    chat_completion = client.chat.completions.create(
        messages=st.session_state.conversation_history,
        model="llama3-8b-8192",
    )
    response = chat_completion.choices[0].message.content

    # Agrega la respuesta del chatbot al historial de conversación
    st.session_state.conversation_history.append({"role": "assistant", "content": response})

    return response

# Configuración de la interfaz de Streamlit
st.title("Chatbot con LLaMA 3")
st.subheader("¡Hazme una pregunta!")

user_input = st.text_input("Usuario:", "")

if user_input:
    response = generate_response(user_input)
    st.write(f"**Chatbot**: {response}")

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Widgets para la interacción
input_box = widgets.Text(
    placeholder='Introduce tu pregunta...',
    description='Pregunta:',
    continuous_update=False  # Asegúrate de deshabilitar actualizaciones continuas
)
output_area = widgets.Output()

def handle_input(change):
    with output_area:
        output_area.clear_output()  # Limpiar salida previa
        query = input_box.value
        if query.strip():  # Evitar procesar entradas vacías
            # Aquí debes integrar las funciones query_pinecone, create_prompt, y ask_groq
            # Esto es un ejemplo básico para mostrar cómo debería verse
            print(f"Consulta procesada: {query}")
            # Ejemplo de respuesta ficticia
            print("Respuesta: Esto es un ejemplo de respuesta generada.")
        else:
            print("Por favor, introduce una pregunta válida.")

# Usar observe para detectar cambios en el valor del campo de texto
input_box.observe(handle_input, names='value')

# Mostrar widgets en el notebook
display(input_box, output_area)
